##Netflix recommendation Engine

## **This will not work in Jupiter notebook as it uses Surprise package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
netflix_dataset = pd.read_csv('combined_data.txt', encoding='utf-8', header = None, names=['Cust_id', 'Rating'], usecols=[0,1])

In [7]:
netflix_dataset

,Cust_id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
24058258,2591364,2.0
24058259,1791000,2.0
24058260,512536,5.0
24058261,988963,3.0


In [9]:
netflix_dataset.isnull().sum()

Cust_id       0
Rating     4499
dtype: int64

In [10]:
movie_count = netflix_dataset.isnull().sum()["Rating"]
movie_count

4499

In [11]:
movie_id = None
movie_np = []

#iterate over the DataFrame rows
for cust_id in netflix_dataset['Cust_id'] :
    if ':' in cust_id :
        #update the current movie ID
        movie_id = int(cust_id.replace(':', ''))
    movie_np.append(movie_id)

In [14]:
# In above, we are fetching the movie ID to list movie_np tp map the movie ID with data frame

In [15]:
netflix_dataset['Movie_Id'] = movie_np
netflix_dataset

,Cust_id,Rating,Movie_Id
0,1:,NaN,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
...,...,...,...
24058258,2591364,2.0,4499
24058259,1791000,2.0,4499
24058260,512536,5.0,4499
24058261,988963,3.0,4499


In [16]:
netflix_dataset = netflix_dataset[netflix_dataset['Rating'].notna()]

In [17]:
netflix_dataset

,Cust_id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1
...,...,...,...
24058258,2591364,2.0,4499
24058259,1791000,2.0,4499
24058260,512536,5.0,4499
24058261,988963,3.0,4499


In [18]:

netflix_dataset['Cust_id'] = netflix_dataset['Cust_id'].astype(int)

C:\Users\Centina\AppData\Local\Temp\ipykernel_7072\2787905673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_dataset['Cust_id'] = netflix_dataset['Cust_id'].astype(int)


In [19]:
netflix_dataset.dtypes

Cust_id       int32
Rating      float64
Movie_Id      int64
dtype: object

In [20]:
# Prefiltering of the movies

In [21]:
dataset_movie_summary=netflix_dataset.groupby('Movie_Id')['Rating'].agg(["count"])

In [22]:
dataset_movie_summary

,count
Movie_Id,
1,547
2,145
3,2012
4,142
5,1140
...,...
4495,614
4496,9519
4497,714


In [23]:
#now we will create a benchmark
movie_benchmark=round(dataset_movie_summary['count'].quantile(0.6),0)
movie_benchmark

908.0

In [24]:
drop_movie_list=dataset_movie_summary[dataset_movie_summary['count']<movie_benchmark].index
drop_movie_list

Index([   1,    2,    4,    7,    9,   10,   11,   12,   13,   14,
       ...
       4480, 4481, 4486, 4487, 4491, 4494, 4495, 4497, 4498, 4499],
      dtype='int64', name='Movie_Id', length=2699)

In [27]:
netflix_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24053764 entries, 1 to 24058262
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_id   int32  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int32(1), int64(1)
memory usage: 642.3 MB


In [28]:
#now we will remove all the users that are in-active
dataset_cust_summary=netflix_dataset.groupby('Cust_id')['Rating'].agg(["count"])
dataset_cust_summary

,count
Cust_id,
6,153
7,195
8,21
10,49
25,4
...,...
2649404,12
2649409,10
2649421,3


In [29]:
cust_benchmark=round(dataset_cust_summary['count'].quantile(0.6),0)
cust_benchmark

36.0

In [31]:
drop_cust_list=dataset_cust_summary[dataset_cust_summary['count'] < cust_benchmark].index
drop_cust_list

Index([      8,      25,      33,      83,      94,     126,     130,     133,
           142,     149,
       ...
       2649337, 2649343, 2649351, 2649376, 2649379, 2649384, 2649401, 2649404,
       2649409, 2649421],
      dtype='int32', name='Cust_id', length=282042)

In [32]:
netflix_dataset=netflix_dataset[~netflix_dataset['Movie_Id'].isin(drop_movie_list)]
netflix_dataset=netflix_dataset[~netflix_dataset['Cust_id'].isin(drop_cust_list)]
print('After the triming, the shape is: {}'.format(netflix_dataset.shape))

After the triming, the shape is: (19695836, 3)


In [33]:
netflix_dataset

,Cust_id,Rating,Movie_Id
696,712664,5.0,3
697,1331154,4.0,3
698,2632461,3.0,3
699,44937,5.0,3
700,656399,4.0,3
...,...,...,...
24056842,1055714,5.0,4496
24056843,2643029,4.0,4496
24056844,267802,4.0,4496
24056845,1559566,3.0,4496


# ** Model Implementation **

In [35]:
df_title = pd.read_csv("movie_titles.csv", encoding='ISO-8859-1', header=None, usecols=[0,1,2], names=["Movie_Id", "Year", "Name"])

In [36]:
df_title

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [37]:
!pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/154.4 kB ? eta -:--:--
     ---- -------------------------------- 20.5/154.4 kB 330.3 kB/s eta 0:00:01
     ----------------- ------------------- 71.7/154.4 kB 787.7 kB/s eta 0:00:01
     -------------------------------------- 154.4/154.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [117 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\surprise
  copying surprise\accuracy.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\dataset.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\dump.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\reader.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\trainset.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\utils.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\__init__.py -> build\lib.win-amd64-cpython-312\surprise
  copying surprise\__main__.py -> build\lib.win-amd64-cpython-312\surprise
  creating build\lib.win-amd64-cpython-312\surprise\model_selection
  copyi

In [38]:
# Supris vill have recommendation required methods

In [39]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import Cross_validate

ModuleNotFoundError: No module named 'surprise'

In [40]:
reader = Reader()

NameError: name 'Reader' is not defined

In [41]:
#we only work with top 100K rows for quick runtime
data=Dataset.load_from_df(netflix_dataset[['Cust_Id','Movie_Id','Rating']][:100000], reader) 

# In above, reader is helping to read each row of my data so that load_from_df can convert it to SVD readable format

NameError: name 'Dataset' is not defined

In [42]:
svd = SVD()

NameError: name 'SVD' is not defined

In [43]:
cross_validate(model, data, measures=['RMSE'], cv=3)

NameError: name 'cross_validate' is not defined

# Reccomendation

In [45]:
#making a copy for each individual customer for the recomendation
user_1331154 = df_title.copy()
user_1331154

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [46]:
user_1331154=user_1331154[~user_1331154['Movie_Id'].isin(drop_movie_list)]
user_1331154

,Movie_Id,Year,Name
2,3,1997.0,Character
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
7,8,2004.0,What the #$*! Do We Know!?
15,16,1996.0,Screamers
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [47]:
user_1331154['Estimate_Score']=user_1331154['Movie_Id'].apply(lambda x: model.predict(1331154, x).est)

NameError: name 'model' is not defined

In [48]:
user_1331154

,Movie_Id,Year,Name
2,3,1997.0,Character
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
7,8,2004.0,What the #$*! Do We Know!?
15,16,1996.0,Screamers
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [49]:
user_1331154.sort_values('Estimate_Score', ascending=False)

KeyError: 'Estimate_Score'

In [50]:
user_1331154.sort_values('Estimate_Score', ascending=False).head(5)

KeyError: 'Estimate_Score'